### First let's get all the vessel labels in the videos

In [1]:
import os

dct = {}
directory = "videos"

for filename in os.listdir(directory):
    if filename.endswith(".mp4"):
        dct[filename.split(' - ')[0].split('Over')[0].split('Under')[0]] = 0

In [2]:
dct

{'Bowriding': 0,
 'CabinCruiser': 0,
 'Commercial': 0,
 'Enforcement': 0,
 'HalfCab': 0,
 'HumanPowered': 0,
 'Open': 0}

### We have 7 different labels, let's move forward to count how many videos do we have for each video label

In [3]:
for filename in os.listdir(directory):
    if filename.endswith(".mp4"):
        dct[filename.split(' - ')[0].split('Over')[0].split('Under')[0]] += 1

dct

{'Bowriding': 10,
 'CabinCruiser': 210,
 'Commercial': 42,
 'Enforcement': 72,
 'HalfCab': 335,
 'HumanPowered': 89,
 'Open': 152}

### Looks like our video counts aren't same for each label, and that imbalance will differently ruin our predictions, let's balance things up and also cut a little bit from each video leading and tailing to avoid frames with no vessels in it!

In [4]:
import cv2

processed = []

for key in dct.keys():
        for n in range(min(dct.values())):
            for filename in os.listdir(directory):
                if filename.endswith(".mp4") and filename.startswith(key) and filename not in processed:
                    
                    file_path = os.path.join(directory, filename)
                    
                    # Open the input video
                    cap = cv2.VideoCapture(file_path)

                    # Get the video fps, width and height
                    fps = cap.get(cv2.CAP_PROP_FPS)
                    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

                    # Calculate the total number of frames
                    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                    # Set the start and end points for the new video
                    start_frame = int(num_frames / 4)
                    end_frame = int(num_frames * 3 / 4)

                    # Create the output video writer
                    out = cv2.VideoWriter(os.path.join('processed videos', filename), cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

                    # Read and write the frames
                    for i in range(num_frames):
                        ret, frame = cap.read()
                        if ret:
                            if i >= start_frame and i < end_frame:
                                out.write(frame)
                        else:
                            break

                    # Release the input and output video objects
                    out.release()
                    print(filename,' Done')
                    processed.append(filename)
                    break

Bowriding - 20230425_104347.mp4  Done
Bowriding - 20230425_111326.mp4  Done
Bowriding - 20230425_113604.mp4  Done
Bowriding - 20230425_135100.mp4  Done
Bowriding - 20230425_140758.mp4  Done
Bowriding - 20230425_144104.mp4  Done
Bowriding - 20230425_144832.mp4  Done
Bowriding - 20230425_154948.mp4  Done
Bowriding - 20230425_170555.mp4  Done
Bowriding - PTZ_motion_20230326_164017.mp4  Done
CabinCruiserOver5Kn - 20230425_125536.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230323_160726.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230323_161821.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230323_161831.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230324_110550.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230325_151407.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230325_162033.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230326_164523.mp4  Done
CabinCruiserOver5Kn - PTZ_motion_20230326_170231.mp4  Done
CabinCruiserOver5Kn - VLHS_20230323_160750.mp4  Done
CommercialOver5kn - PTZ_motion_20230

### Now let's make sure that we've balanced everything!

In [5]:
dct_ = {}
directory = "processed videos"

for filename in os.listdir(directory):
    if filename.endswith(".mp4"):
        dct_[filename.split(' - ')[0].split('Over')[0].split('Under')[0]] = 0
for filename in os.listdir(directory):
    if filename.endswith(".mp4"):
        dct_[filename.split(' - ')[0].split('Over')[0].split('Under')[0]] += 1
dct_

{'Bowriding': 10,
 'CabinCruiser': 10,
 'Commercial': 10,
 'Enforcement': 10,
 'HalfCab': 10,
 'HumanPowered': 10,
 'Open': 10}

### Great, let's now start extarcting frames from the videos, this function extracts frames from a video file and saves them as individual image files in a specified output directory.

### The function takes four arguments:

- pathIn: the path to the input video file.
- imname: the base name to use for the output image files.
- pathOut: the path to the output directory where the extracted images will be saved.
- num_ms: the time interval in milliseconds between each frame to extract (default is 100ms).
### The function uses OpenCV's VideoCapture object to read in the video file. It then sets the position of the VideoCapture object to count * num_ms milliseconds and reads in the frame at that position using vidcap.read(). It repeats this process until either it reaches the end of the file or until it has extracted a maximum of 40 frames. For each successfully extracted frame, the function writes the image to the output directory with a unique file name generated using uuid.uuid4(). The function prints the number of frames successfully extracted before reaching the end of the file.

In [6]:
import uuid


def extractImages(pathIn, imname, pathOut, num_ms=1000): #num_ms=1000 means that we're extracting only one frame per second
    vidcap = cv2.VideoCapture(pathIn)
    success, image = vidcap.read()
    success = True
    count = 0
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*num_ms))
        success, image = vidcap.read()
        if success:
            impath = os.path.join(pathOut, f'{imname}-' + str(uuid.uuid4()) + ".jpg")
            cv2.imwrite(impath, image)
        count += 1
    else:
        print(f"Read out {count} frames before end of file.")

In [7]:
directory = "processed videos"

for filename in os.listdir(directory):
            print(filename)
            file_path = os.path.join(directory, filename)
            extractImages(file_path,filename.split(' - ')[0].split('Over')[0].split('Under')[0],'images')

Bowriding - 20230425_104347.mp4
Read out 5 frames before end of file.
Bowriding - 20230425_111326.mp4
Read out 5 frames before end of file.
Bowriding - 20230425_113604.mp4
Read out 5 frames before end of file.
Bowriding - 20230425_135100.mp4
Read out 5 frames before end of file.
Bowriding - 20230425_140758.mp4
Read out 5 frames before end of file.
Bowriding - 20230425_144104.mp4
Read out 5 frames before end of file.
Bowriding - 20230425_144832.mp4
Read out 5 frames before end of file.
Bowriding - 20230425_154948.mp4
Read out 4 frames before end of file.
Bowriding - 20230425_170555.mp4
Read out 5 frames before end of file.
Bowriding - PTZ_motion_20230326_164017.mp4
Read out 5 frames before end of file.
CabinCruiserOver5Kn - 20230425_125536.mp4
Read out 5 frames before end of file.
CabinCruiserOver5Kn - PTZ_motion_20230323_160726.mp4
Read out 4 frames before end of file.
CabinCruiserOver5Kn - PTZ_motion_20230323_161821.mp4
Read out 4 frames before end of file.
CabinCruiserOver5Kn - PTZ_m

In [2]:
import cv2
import os
import uuid

def extractMiddleFrame(pathIn, imname, pathOut):
    vidcap = cv2.VideoCapture(pathIn)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    middle_frame = total_frames // 2
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)
    success, image = vidcap.read()
    if success:
        impath = os.path.join(pathOut, f'{imname}-' + str(uuid.uuid4()) + ".jpg")
        cv2.imwrite(impath, image)
        print(f"Extracted middle frame and saved as {impath}")
    else:
        print("Error: Could not read middle frame")

directory = "videos"

for filename in os.listdir(directory):
            print(filename)
            file_path = os.path.join(directory, filename)
            extractMiddleFrame(file_path,filename.split(' - ')[0].split('Over')[0].split('Under')[0],'middle_images')

Bowriding - 20230425_104347.mp4
Extracted middle frame and saved as middle_images\Bowriding-74bf167d-f8c2-4c25-959e-8fe240289728.jpg
Bowriding - 20230425_111326.mp4
Extracted middle frame and saved as middle_images\Bowriding-c9e8caed-28b6-4a46-b67c-d14ceaa4b3d2.jpg
Bowriding - 20230425_113604.mp4
Extracted middle frame and saved as middle_images\Bowriding-306c3c18-77b2-449d-a442-693159a822b3.jpg
Bowriding - 20230425_135100.mp4
Extracted middle frame and saved as middle_images\Bowriding-7d75d48e-fc9c-43cd-9fb4-6823835844cc.jpg
Bowriding - 20230425_140758.mp4
Extracted middle frame and saved as middle_images\Bowriding-4c944dbc-ac7c-4162-a46a-23fe922c03a8.jpg
Bowriding - 20230425_144104.mp4
Extracted middle frame and saved as middle_images\Bowriding-1124bcfa-e2e3-448f-beae-3aa79d6ff8ad.jpg
Bowriding - 20230425_144832.mp4
Extracted middle frame and saved as middle_images\Bowriding-5f74bb60-23c5-4535-837b-70f21de7366e.jpg
Bowriding - 20230425_154948.mp4
Extracted middle frame and saved as m

### Now, for the final step in our data-extracting process, let's split the images we've extracted from the videos into a training and validation sets, each set will have 7 labels that are the vessels types from the videos.

In [1]:
import shutil
import random

# Set the paths for the input and output folders
input_folder = "images"
train_folder = "train"
val_folder = "validation"

# Set the percentage of images for validation
val_percent = 0.2

# Loop through the files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):  # Only process image files
        label = filename.split('-')[0].lower()  # Determine the label based on the file name
        src_path = os.path.join(input_folder, filename)  # Get the source path of the image
        if random.random() < val_percent:  # Randomly assign images to either training or validation folder
            dst_folder = os.path.join(val_folder, label)  # Set the destination folder for validation
        else:
            dst_folder = os.path.join(train_folder, label)  # Set the destination folder for training
        os.makedirs(dst_folder, exist_ok=True)  # Create the destination folder if it doesn't exist
        dst_path = os.path.join(dst_folder, filename)  # Get the destination path for the image
        shutil.copy(src_path, dst_path)  # Copy the image to the destination folder

FileNotFoundError: [Errno 2] No such file or directory: 'images\\CabinCruiser-1170bb3a-4cef-4060-961b-41eb410d35b1.txt'